In [5]:
import open3d as o3d
import numpy as np
import torch

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# # pcd to npy
---

In [6]:
# device = o3d.core.Device('cuda:1')
pcd_f = o3d.t.io.read_point_cloud('/data/NIA50/data/2-050_sensor_sample/lidar/1639543825.777566671.pcd')
print(pcd_f)

positions = pcd_f.point.positions.numpy()
intensity = pcd_f.point.intensity.numpy()
pcd = np.concatenate((positions, intensity), axis = 1)

pcd

PointCloud on CPU:0 [276640 points (Float32)].
Attributes: intensity (dtype = Float32, shape = {276640, 1}).


array([[ -1.5803028 ,  -9.255083  ,  -2.3824618 ,   6.        ],
       [ -2.5670419 , -16.682653  ,  -2.200407  ,   1.        ],
       [ -3.6694992 , -28.680662  ,  -1.4689597 ,   0.        ],
       ...,
       [ -0.        ,   0.        ,   0.        ,   2.        ],
       [  0.        ,   0.        ,   0.        ,   9.        ],
       [ -0.7640565 ,  38.062027  ,  -0.88586044,   6.        ]],
      dtype=float32)

In [ ]:
import pickle as pkl

custom_infos_train = '/data/NIA50/OpenPCD/data/custom/custom_infos_train.pkl'
with open(custom_infos_train, 'rb') as f:
    train_pkl = pkl.load(f)